In [ ]:
!pip install anthropic pandas jupyter openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import random
import openai
import anthropic
import pandas as pd
import os
import glob
import re
import time
from google.colab import userdata

In [ ]:
# Verify and set the API key
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key is None or openai_api_key.strip() == "":
    raise ValueError("OpenAI API key is not set. Please set it in Colab secrets.")

openai.api_key = openai_api_key
print("OpenAI API key set successfully.")

OpenAI API key set successfully.


In [ ]:
def generate_documentation_and_prompt(test_case_name, test_case_type, file_path):
    # Extract application and directory context from the file path
    parts = file_path.split('/')
    application_name = parts[-3]  # Adjust the index based on your specific path structure
    directory_name = parts[-2]

    # Define actions and types for variability in the prompt
    actions = ['Generate', 'Create', 'Build', 'Validate']
    types = ['keyword-driven', 'data-driven']

    # Select a random action
    action = random.choice(actions)
    type_phrase = 'keyword-driven' if 'Keyword' in directory_name else 'data-driven'

    # Construct the full prompt using the extracted context
    prompt = f"{action} {type_phrase} {application_name.lower()} for {directory_name.replace('_', ' ')} {test_case_name}"

    # Generate documentation
    documentation_template = f"Provide a summary for the test case '{test_case_name}' explaining its purpose"
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": documentation_template}],
            max_tokens=150
        )
        documentation = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating documentation for {test_case_name}: {str(e)}")
        documentation = f"Documentation for {test_case_name} could not be generated due to an error."

    return prompt, documentation

In [ ]:
def infer_return_type(keyword_name, args):
    keyword_name = keyword_name.lower()
    args = str(args).lower()

    if keyword_name.startswith("validate") or "is displayed" in keyword_name:
        return "${boolean}"

    if any(word in keyword_name for word in ["count", "number", "total", "sum", "calculate"]):
        if "float" in args or "." in args:
            return "${float}"
        else:
            return "${integer}"

    if any(word in keyword_name for word in ["list", "array", "multiple", "all"]):
        return "@{list}"

    if any(word in keyword_name for word in ["dict", "map", "hash", "object"]):
        return "&{dictionary}"

    return "${string}"

In [ ]:
def process_keyword_driven_test_case(file_path):
    test_case_name = os.path.splitext(os.path.basename(file_path))[0]
    test_case_data = pd.read_csv(file_path)

    keywords = test_case_data['KeywordName'].tolist()
    if 'Arguments' in test_case_data.columns:
        args_list = test_case_data['Arguments'].tolist()
    else:
        args_list = test_case_data[['Argument1', 'Argument2', 'Argument3']].fillna('').apply(
            lambda row: '    '.join(filter(None, row)), axis=1).tolist()

    # prompt, documentation = generate_documentation_and_prompt(test_case_name)
    # Get dynamic prompt and documentation based on file path
    prompt, documentation = generate_documentation_and_prompt(test_case_name, 'keyword-driven', file_path)

    settings = (
        "*** Settings ***\n"
        f"Documentation    Validate {test_case_name}\n"
        "Test Setup       Initialize Environment\n"
        "Test Teardown    Cleanup Environment\n"
        "Suite Setup      Open Application\n"
        "Suite Teardown   Close Application\n"
        "Resource         resource_file.robot\n\n"
    )
    variables = (
        "*** Variables ***\n"
        "${VALID_USER}       testuser\n"
        "${VALID_PASSWORD}   secret\n\n"
    )
    test_cases = f"*** Test Cases ***\n{test_case_name}\n    [Documentation]    Validate {test_case_name} using the following steps\n"
    test_cases += '    \n    '.join(f"{kw}    {args}" for kw, args in zip(keywords, args_list)) + "\n\n"

    keywords_section = "*** Keywords ***\n"
    for keyword, args in zip(keywords, args_list):
        return_type = infer_return_type(keyword, args)
        keywords_section += (
            f"{keyword}\n"
            f"    [Arguments]    {args}\n"
            f"    [Return]    {return_type}\n"
            "    [Documentation]    Keyword description\n"
            f"    Log    {args}\n\n"
        )

    refined_response = settings + variables + test_cases + keywords_section

    return [{"prompt": prompt, "testcase_type": "keyword-driven", "testcase_name": test_case_name, "response": refined_response, "documentation": documentation}]


In [ ]:
def process_data_driven_test_case(file_path):
    test_case_name = os.path.splitext(os.path.basename(file_path))[0]
    test_case_data = pd.read_csv(file_path)

    args1 = test_case_data['Argument1'].tolist() if 'Argument1' in test_case_data.columns else [''] * len(test_case_data)
    args2 = test_case_data['Argument2'].tolist() if 'Argument2' in test_case_data.columns else [''] * len(test_case_data)
    args3 = test_case_data['Argument3'].tolist() if 'Argument3' in test_case_data.columns else [''] * len(test_case_data)

    keywords = test_case_data['KeywordName'].tolist()

    all_arguments = [
        '    '.join(filter(None, [f"${{{arg}}}" for arg in [a1, a2, a3]]))
        for a1, a2, a3 in zip(args1, args2, args3)
    ]

    prompt, documentation = generate_documentation_and_prompt(test_case_name, 'data-driven', file_path)
    # prompt, documentation = generate_documentation_and_prompt(test_case_name)

    settings = (
        "*** Settings ***\n"
        f"Documentation    Validate {test_case_name}\n"
        "Test Template    Test Execution\n"
        "Resource         resource_file.robot\n\n"
    )
    variables = (
        "*** Variables ***\n"
        "${VALID_USER}       testuser\n"
        "${VALID_PASSWORD}   secret\n\n"
    )
    test_cases = f"*** Test Cases ***\n{test_case_name}\n    [Documentation]    Validate {test_case_name} using the following steps\n"
    test_cases += '    \n    '.join(f"{keywords[i]}    {all_arguments[i]}" for i in range(len(keywords))) + "\n\n"

    keywords_section = "*** Keywords ***\n"
    for i, keyword in enumerate(keywords):
        return_type = infer_return_type(keyword, all_arguments[i])
        keywords_section += (
            f"{keyword}\n"
            f"    [Arguments]    {all_arguments[i]}\n"
            f"    [Return]    {return_type}\n"
            "    [Documentation]    Keyword description\n"
            f"    Log    {all_arguments[i]}\n\n"
        )

    refined_response = settings + variables + test_cases + keywords_section

    return [{"prompt": prompt, "testcase_type": "data-driven", "testcase_name": test_case_name, "response": refined_response, "documentation": documentation}]

In [ ]:
def find_csv_files(root_directories):
    csv_files = []
    for root_directory in root_directories:
        for subdir, dirs, files in os.walk(root_directory):
            for file in glob.glob(os.path.join(subdir, '*.csv')):
                csv_files.append(file)
    return csv_files

In [ ]:
def process_files(file_paths, process_function):
    results = []
    start_time = time.time()
    for file_path in file_paths:
        try:
            result = process_function(file_path)
            results.extend(result)  # Flatten the list of results
        except Exception as e:
            print(f"Failed to process {file_path}: {str(e)}")
    end_time = time.time()
    print(f"Processed {len(file_paths)} files in {end_time - start_time:.2f} seconds")
    return results

In [ ]:
# Root directories for keyword-driven and data-driven test cases
roots_keyword_driven = [
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/API',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/CommonKeywordsForAllMerchants',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/Omnicorp/Customers',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Customers',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Dashboard',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Invoices',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Settings',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Transactions',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Keywords Ram/ClientCentral_Keywords/Web/SterlingCooper/Virtual_Terminal',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/API',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/Web',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW_Keywords Ramashanker/PayNOW_Keywords/Web/PayNOW',
]

roots_data_driven = [
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Autopay Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Omnicorp Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/SchedulePayment Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Sterlingcooper Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/non integrated Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Datadriver Testcases Ramashanker',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/all_transactions_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/autopay_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/customers_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/dashboard_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/failed_login_attempts_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/invoices_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/payment_status_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/scheduled_payments_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/settings_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/tokens_Testcases',
    '/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/ClientCentral_Testcases Ram/ClientCentral_Testcases/virtual_terminal_Testcases'
]

In [ ]:
# Collect all relevant CSV files
keyword_driven_files = find_csv_files(roots_keyword_driven)
data_driven_files = find_csv_files(roots_data_driven)

print(f"Found {len(keyword_driven_files)} keyword-driven test cases")
print(f"Found {len(data_driven_files)} data-driven test cases")

Found 38 keyword-driven test cases
Found 343 data-driven test cases


In [ ]:
# Process keyword-driven and data-driven test cases
refined_KD_data_openai = process_files(keyword_driven_files, process_keyword_driven_test_case)
refined_DD_data_openai = process_files(data_driven_files, process_data_driven_test_case)

Processed 38 files in 96.32 seconds
Failed to process /content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/SDT UAR Test Cases/PayNOW Test Cases Ramashanker/PayNOW Test Cases/Sterlingcooper Testcases/TC_50 Validate Customer Is Able To Cancel Login With Invoice Pop-Up.csv: 'utf-8' codec can't decode byte 0x91 in position 102: invalid start byte
Processed 343 files in 783.72 seconds


In [ ]:
# Convert to DataFrame and Save refined data to CSV
keydriven_df = pd.DataFrame(refined_KD_data_openai)
ddriven_df = pd.DataFrame(refined_DD_data_openai)
keydriven_df.to_csv('/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/newKeyword_driven_testcases.csv', index=False)
ddriven_df.to_csv('/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/newData_driven_testcases.csv', index=False)

# print("Keyword-driven test cases saved to keydriven_df.csv")
# print("Data-driven test cases saved to ddriven_df.csv")

In [ ]:
keydriven_df.shape

(38, 5)

In [ ]:
ddriven_df.shape

(342, 5)

In [ ]:
keydriven_df.head()

,prompt,testcase_type,testcase_name,response,documentation
0,Validate data-driven clientcentral_keywords fo...,keyword-driven,common,*** Settings ***\nDocumentation Validate co...,"The purpose of the test case named ""common"" ca..."
1,Validate data-driven clientcentral_keywords fo...,keyword-driven,customer,*** Settings ***\nDocumentation Validate cu...,The test case 'customer' likely serves several...
2,Validate data-driven clientcentral_keywords fo...,keyword-driven,invoice,*** Settings ***\nDocumentation Validate in...,The purpose of the 'invoice' test case is to e...
3,Generate data-driven clientcentral_keywords fo...,keyword-driven,processpayment,*** Settings ***\nDocumentation Validate pr...,The purpose of the test case 'processpayment' ...
4,Create data-driven clientcentral_keywords for ...,keyword-driven,token,*** Settings ***\nDocumentation Validate to...,A test case for 'token' typically aims to veri...


In [ ]:
ddriven_df.head()

,prompt,testcase_type,testcase_name,response,documentation
0,Build data-driven paynow test cases for Autopa...,data-driven,TC_01 Validate Customer Is Able To Schedule Au...,*** Settings ***\nDocumentation Validate TC...,The test case 'TC_01 Validate Customer Is Able...
1,Validate data-driven paynow test cases for Aut...,data-driven,TC_02 Validate Customer Is Able To Schedule Au...,*** Settings ***\nDocumentation Validate TC...,The purpose of the test case 'TC_02 - Validate...
2,Create data-driven paynow test cases for Autop...,data-driven,TC_03 Validate Customer Is Able To Schedule Au...,*** Settings ***\nDocumentation Validate TC...,The test case 'TC_03 Validate Customer Is Able...
3,Generate data-driven paynow test cases for Aut...,data-driven,TC_04 Validate Customer Is Able To Delete Save...,*** Settings ***\nDocumentation Validate TC...,"The test case ""TC_04 Validate Customer Is Able..."
4,Build data-driven paynow test cases for Autopa...,data-driven,TC_05 Validate Customer Is Able To Close Delet...,*** Settings ***\nDocumentation Validate TC...,The purpose of the test case 'TC_05 Validate C...


In [ ]:
keydriven_df.to_excel('/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/newKeyword_driven_testcases.xlsx', index=False)
ddriven_df.to_excel('/content/drive/MyDrive/WORKS/SOFTWARE-DRIVEN-TEST AUTOMATION/dataset & info/newData_driven_testcases.xlsx', index=False)

print("done!!!")

done!!!
